In [1]:
import os
import torch
import torch.nn as nn
from utility import train_test_split
from numpy import unique
from utility import load_data, batch_training_generator
from torch_utility import accuracy, fully_connected, Flatten
from sequential import Sequential

In [2]:
image_dir = os.getcwd() + '/grey_images_32' 
X_raw, labels = load_data(image_dir)

In [3]:
length, width = X_raw[0].shape
X = torch.Tensor(X_raw).type(torch.float32)
X = X.view(-1, 1, length, width)
X /= 255
unique_y = unique(labels)
y_dim = len(unique_y)
num_unique_labels = len(unique_y)
mapping = {label: i for i, label in enumerate(unique_y)}
numeric_labels = torch.Tensor([mapping[label] for label in labels]).type(torch.long)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, numeric_labels, test_size=0.1,
                                                    shuffle=False)

In [17]:
N = len(X_train)

# 32 X 32 X 1
layers = [nn.Conv2d(1, 6, 5), # 28 X 28 X 32
          nn.ReLU(),
          nn.MaxPool2d(2, 2), # 14 X 14 X 32
          Flatten(),
          nn.Linear(14 * 14 * 6, 14 * 14 * 6),
          nn.ReLU(),
          nn.Linear(14 * 14 * 6, y_dim)
         ]
model = Sequential(*layers)
model

Sequential(
  (layers): ModuleList(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Flatten()
    (4): Linear(in_features=1176, out_features=1176, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1176, out_features=29, bias=True)
  )
)

In [18]:
def entropy_train(model)
epochs = 10
batch_size = 64
op = torch.optim.Adam(model.parameters(), lr=.001)
loss_fn = nn.CrossEntropyLoss(reduction='sum')
for epoch_num in range(epochs):
    epoch_loss = 0
    training = batch_training_generator(X_train, y_train, batch_size,
                                        shuffle=True)
    for x_set, y_set in training:
        op.zero_grad()
        output = model(x_set)
        loss = loss_fn(output, y_set)
        loss /= N
        epoch_loss += float(loss)
        loss.backward()
        op.step()
    print(epoch_num, epoch_loss, accuracy(model, X_train, y_train), accuracy(model, X_test, y_test))

0 1.2058014557260321 0.8569731800766284 0.8489655172413794
1 0.27443375160510186 0.9478544061302682 0.9394252873563218
2 0.13408954219448788 0.9686973180076628 0.9605747126436781
3 0.08418097798585222 0.9752234993614304 0.9691954022988506
4 0.06827845129441812 0.9742784163473819 0.9681609195402299
5 0.04558094807350699 0.987816091954023 0.9845977011494252
6 0.0420082114452498 0.9934610472541507 0.9916091954022989
7 0.0377512392063295 0.991698595146871 0.9881609195402299
8 0.03542796586714303 0.9955938697318008 0.9939080459770115
9 0.026145958922029422 0.9774457215836526 0.9725287356321839
